In [10]:
import pandas as pd
import numpy as np
from comet_ml import API


## Preprocess the data

In [6]:
test_data = pd.read_csv("test_2019.csv")
test_data = test_data.drop(labels='Unnamed: 0', axis=1).reset_index(drop=True)
test_data.tail()

,event_idx,period_time,period,game_id,team_away_name,team_home_name,is_goal,coordinate_x,coordinate_y,shot_type,...,number_friendly,number_opposing,power_play,distance,from_last_distance,angle,last_angle,time_from_last,change_angle,speed
73862,319,16:01,3,2019020001,Ottawa Senators,Toronto Maple Leafs,0,-59.0,-24.0,Snap Shot,...,5,4,34,38.4187,123.016259,0.6747,0.8050,61,0.0,0.0
73863,326,17:20,3,2019020001,Ottawa Senators,Toronto Maple Leafs,0,-81.0,3.0,Wrist Shot,...,5,4,113,8.5440,3.162278,0.3588,0.5880,20,0.0,0.0
73864,333,18:50,3,2019020001,Ottawa Senators,Toronto Maple Leafs,0,34.0,-6.0,Snap Shot,...,5,5,0,55.3263,58.855756,0.1087,1.3854,50,0.0,0.0
73865,341,19:27,3,2019020001,Ottawa Senators,Toronto Maple Leafs,0,31.0,10.0,Wrist Shot,...,5,5,0,58.8558,49.678969,0.1707,0.8330,27,0.0,0.0
73866,343,19:45,3,2019020001,Ottawa Senators,Toronto Maple Leafs,0,77.0,0.0,Wrist Shot,...,5,5,0,12.0000,9.899495,0.0000,0.9505,45,0.0,0.0


In [9]:
test_data['game_id'] = test_data['game_id'].astype(str)
regular_ds = test_data[test_data.game_id.str.contains('^201902')]
playoffs_ds = test_data[test_data.game_id.str.contains('^201903')]

## 7.1

In [ ]:
def get_registered_comet_model(workspace, model, version, output_path):
    api = API()
    # Download a Registry Model
    file = api.download_registry_model({workspace}, model, version, output_path=output_path, expand=True)
    return file

In [19]:
api = API(api_key="c2REbE8eQaoRTP059ajV8VYn9")
rl_angle_model = api.download_registry_model('ift-6758-2',"lr-angle",'1.0.0', output_path = './',expand=True)
rl_distance_model = api.download_registry_model('ift-6758-2',"lr-distance",'1.0.0', output_path = './',expand=True)
rl_angle_distance_model = api.download_registry_model('ift-6758-2',"lr-distance-angle",'1.0.0', output_path = './',expand=True)
xgb_best = api.download_registry_model('ift-6758-2',"best-treebased-5-3-h5",'1.0.0', output_path = './',expand=True)
#question 6

COMET INFO: Downloading registry model 'lr-angle', version '1.0.0', stage None from workspace 'ift-6758-2'...
COMET INFO: Unzipping model to '/Users/minaparham/Desktop/course/Data science/IFT6758-Project/notebooks' ...
COMET INFO: done!
COMET INFO: Downloading registry model 'lr-distance', version '1.0.0', stage None from workspace 'ift-6758-2'...
COMET INFO: Unzipping model to '/Users/minaparham/Desktop/course/Data science/IFT6758-Project/notebooks' ...
COMET INFO: done!
COMET INFO: Downloading registry model 'lr-distance-angle', version '1.0.0', stage None from workspace 'ift-6758-2'...
COMET INFO: Unzipping model to '/Users/minaparham/Desktop/course/Data science/IFT6758-Project/notebooks' ...
COMET INFO: done!
COMET INFO: Downloading registry model 'best-treebased-5-3-h5', version '1.0.0', stage None from workspace 'ift-6758-2'...
COMET INFO: Unzipping model to '/Users/minaparham/Desktop/course/Data science/IFT6758-Project/notebooks' ...
COMET INFO: done!


In [21]:
print(rl_angle_distance_model)

None
